In [45]:
sqlContext = HiveContext(sc)

id_domain = sqlContext.sql("SELECT id, url_domain, hits FROM user_mromanov.user_domain_ordered_10_8")
sc.version

u'1.3.0'

In [46]:
# id_track = id_domain\
#     .map(lambda x: ((x.id, x.url_domain), x.hits))\
#     .reduceByKey(lambda a, b: a + b)\
#     .filter(lambda x: x[1] > 3)
    
# ОТБИРАЕМ ВСЕ КУКИ, ДЛИНА ТРЕКА ДОЛЖНА БЫТЬ БОЛЬШЕ, ЧЕМ 3
ids = id_domain\
    .map(lambda x: (x.id, 1))\
    .reduceByKey(lambda a, b: a + b)\
    .filter(lambda x: x[1] >= 3)\
    .map(lambda x: x[0])

# ОТБИРАЕМ ДОМЕНЫ, КОЛИЧЕСТВО УНИКАЛЬНЫХ КУК ДОЛЖНО БЫТЬ БОЛЬШЕ 100
urls = id_domain\
    .map(lambda x: (x.url_domain, 1))\
    .reduceByKey(lambda a, b: a + b)\
    .filter(lambda x: x[1] >= 100)\
    .map(lambda x: x[0])

ids_collection = ids.collect()
urls_collection = urls.collect()

ids_to_index = {}
for index in range(len(ids_collection)):
    ids_to_index[ids_collection[index]] = index
    
urls_to_index = {}
for index in range(len(urls_collection)):
    urls_to_index[urls_collection[index]] = index

print len(ids_to_index)
print len(urls_to_index)

MemoryError: 

In [ ]:
id_track = id_domain\
    .map(lambda x: (x.id, x.url_domain, float(x.hits)))\
    .filter(lambda x: (x[1] in urls_to_index))\
    .filter(lambda x: (x[0] in ids_to_index))\
    .map(lambda x: ((ids_to_index[x[0]], urls_to_index[x[1]]), x[2]))\
    .reduceByKey(lambda a, b: a + b)

coordinates = id_track.collect()

# print coordinates[:100]

coordinates_new = []
num_rows = 0
num_cols = 0

print len(coordinates)

for index in range(len(coordinates)):
    row_idx = coordinates[index][0][0]
    column_idx = coordinates[index][0][1]
    value = float(coordinates[index][1])
    
    if row_idx >= num_rows:
        num_rows = row_idx + 1
    
    if column_idx >= num_cols:
        num_cols = column_idx + 1
    
    coordinates_new.append([row_idx, column_idx, value])

In [41]:
import pyspark.mllib.linalg
import sklearn.preprocessing

user_domain = pyspark.mllib.linalg.scipy.sparse.lil_matrix((num_rows, num_cols), dtype=float)

print user_domain.shape

for instance in coordinates_new:
#     print instance[0], instance[1]
    user_domain[instance[0], instance[1]] = instance[2]

user_domain = sklearn.preprocessing.normalize(user_domain, norm='l2', axis=1)
    
v, s, u = pyspark.mllib.linalg.scipy.sparse.linalg.svds(user_domain.T, k=100)

v = v.T
u = u.T

print s

# pyspark.mllib.scipy.sparse.

# pyspark.ml.feature
# pyspark.mllib.feature
# pyspark.mllib.linalg.scipy.linalg.svd

# import scipy.sparse
# import pyspark.mllib.linalg
# pyspark.mllib.linalg.
# entries = sc.parallelize(coordinates_new)

# spark 
# sc.version

(936901, 8625)
[  28.88046038   28.97614763   29.01014713   29.04480929   29.09222675
   29.63171081   29.653766     29.78464213   29.86496456   30.09846518
   30.16049474   30.32300455   30.6782987    30.75423751   30.92871121
   31.1069398    31.27795476   31.51164268   31.55705394   31.83311033
   31.85394408   32.51840788   32.66612255   32.7400982    33.18307202
   33.3798739    33.41751243   33.72227104   34.09655476   34.22774228
   34.34374188   34.47290445   34.53304622   34.63839667   34.83764082
   34.94412118   35.47433179   35.62911096   36.11289567   36.12947828
   36.22468509   36.26870473   36.69899916   37.00280491   37.18016228
   37.32890317   37.64758124   38.15822586   38.24172381   38.27131662
   38.72949339   38.88057536   39.07689099   39.48122786   39.76058102
   40.18595545   41.57960009   42.05099186   42.08364801   42.28956173
   42.74052442   43.00097264   43.23488486   44.47753027   45.20840984
   46.34055459   46.41809094   47.29838904   47.49464025   47.

In [42]:
print v.shape
print u.shape

for index in range(v.shape[0])[::-1][:20]:
    singular_vector = v[index, :]
    order = singular_vector.argsort()[::-1]
    print '=============================Singular vector #' + str(index)
    for idx in order[:20]:
        print urls_collection[idx], singular_vector[idx]

(100, 8625)
(936901, 100)
=============================Singular vector #99
seucorpoperfeito.com.br 1.96070907131e-17
npkpmz.ru 1.42817534242e-17
webhoroscopo.com 1.13728979835e-17
cntronics.com 6.33900404828e-18
online-seriale.ru -3.77255939452e-18
projectsion.ru -5.21063474145e-18
sulla-salute.com -6.56884309893e-18
liveinternet.club -8.04841944599e-18
huanpornos.com -8.40677900116e-18
sexxxok.com -1.29125273716e-17
kinorur.ru -1.4523796332e-17
lasaludfamiliar.com -1.47784686772e-17
sessovideosgratiss.com -1.69371437686e-17
forklmod.at.ua -2.68546265352e-12
smarton.net.ua -3.49203793221e-12
evdekieczane.net -4.48473596694e-12
krd.sputniknews.com -6.62638399997e-12
sexonline.sex.pl -2.93324919992e-11
habersirasi.com -3.58207486083e-11
guidewhat.com -3.90960935601e-11
=============================Singular vector #98
e.mail.ru 0.678672090383
ok.ru 0.635743029923
mail.ru 0.277330026324
avito.ru 0.052531643805
go.mail.ru 0.0443778367145
my.mail.ru 0.0348304370877
news.mail.ru 0.02956607655

In [44]:

import numpy
import sklearn.preprocessing

new_features = user_domain.T.dot(u).T

user_domain_normed = sklearn.preprocessing.normalize(user_domain, norm='l2', axis=0)
new_features_normed = sklearn.preprocessing.normalize(new_features, norm='l2', axis=0)

print user_domain_normed.shape
print new_features_normed.shape

# url_distances = user_domain_normed.T.dot(user_domain_normed).toarray()
# new_url_distances = new_features_normed.T.dot(new_features_normed)

for url_id in range(user_domain_normed.shape[1])[:100]:
    
    current_domain = user_domain_normed[:, url_id]
    current_new_features = new_features_normed[:, url_id]
    
    distances = user_domain_normed.T.dot(current_domain).toarray()
    new_distances = new_features_normed.T.dot(current_new_features)
    
    order = new_distances.flatten().argsort()[::-1]
    
    print '===========Current URL:', urls_collection[url_id]
    for index in order[:20]:
        print urls_collection[index], '\t\t', distances[index], '\t\t', new_distances[index]
        
# for url_id in range(user_domain_normed.shape[1])[:100]:
    
#     current_domain = user_domain_normed[:, url_id]
#     current_new_features = new_features_normed[:, url_id]
    
#     distances = user_domain_normed.T.dot(current_domain).toarray()
#     new_distances = new_features_normed.T.dot(current_new_features)
    
#     order = new_distances.flatten().argsort()[::-1]
    
#     print '===========Current URL:', urls_collection[url_id]
#     for index in order[:20]:
#         print urls_collection[index], '\t\t', distances[index], '\t\t', new_distances[index]

# for url_id in range(len(urls_collection[:10])):
#     url_features = user_domain[:, url_id]
#     new_url_features = new_features[:, url_id]

#     print numpy.sqrt(url_features.T.dot(url_features).todense())[0, 0]
#     print '=============== URL:', urls_collection[url_id]
    
    
    
#     order = new_url_distances.argsort()[::-1]
#     print order
#     print 'URL name', 'Old cosine', 'New cosine'
#     for index in order[:20]:
#         print index
#         print urls_collection[index], '\t\t', url_distances[index], '\t\t', new_url_distances[index]

(936901, 8625)
(100, 8625)
===========Current URL: 
 		[ 1.] 		1.0
vz.ru 		[ 0.03263741] 		0.914432478555
kazanfirst.ru 		[  5.75189660e-05] 		0.887420930304
x-true.info 		[ 0.00019023] 		0.827240235372
km.ru 		[ 0.00182624] 		0.82049200203
36on.ru 		[ 0.] 		0.81731611302
inosmi.ru 		[ 0.00144633] 		0.817103555055
finobzor.ru 		[  7.98763193e-05] 		0.808289872428
stoletie.ru 		[ 0.00027166] 		0.775108161091
svpressa.ru 		[ 0.00366581] 		0.766161583935
vpk.name 		[ 0.00277471] 		0.763367423176
imperiyanews.ru 		[  1.39510414e-06] 		0.758226735825
politobzor.net 		[ 0.00019141] 		0.757038826436
oko-planet.su 		[ 0.00105474] 		0.743484357094
topwar.ru 		[ 0.00155592] 		0.732524728699
pravdoryb.info 		[ 0.00035926] 		0.729235548707
3mv.ru 		[ 0.00044676] 		0.723186895456
in24.org 		[ 0.] 		0.716199403294
lavaplace.com 		[ 0.] 		0.716057518784
odnako.org 		[  1.78490526e-05] 		0.713423757067
===========Current URL: waysee.ru
waysee.ru 		[ 1.] 		1.0
timeio.ru 		[ 0.97005484] 		0.998242542823